In [1]:
import tensorflow as tf
import keras

In [2]:
import os
import numpy as np

In [3]:
tf.keras.backend.clear_session()

In [4]:
!nvidia-smi

Mon Sep  6 20:00:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| N/A   52C    P0    31W /  N/A |    572MiB /  7973MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [6]:
from preparator import ImageDetectionController

In [7]:
detection_controller = ImageDetectionController()

In [8]:
from classJson import JsonData

CONFIG_PATH = "./conf/conf_ct131405.json"

conf = JsonData(name=CONFIG_PATH)
data_matrix = conf.return_matrix()
matrix = data_matrix["matrix"]
max_height = data_matrix["maxHeight"]
max_width = data_matrix["maxWidth"]

In [9]:
from preparator import ImageDrawer

image_drawer = ImageDrawer(
    detection_controller=detection_controller,
    matrix=matrix,
    max_height=max_height,
    max_width=max_width
)

In [10]:
!ls /tf/mmk_smoke_detection/model

hatches.h5  hatches.h5.dvc


In [11]:
PATH_TO_MODEL = "/tf/mmk_smoke_detection/model/hatches.h5"
vanilla_model = keras.models.load_model(PATH_TO_MODEL, compile=True)

In [12]:
from preparator import VanillaDetector
import glob

PATH_TO_IMAGES = "/tf/mmk_smoke_detection/flow_data/172.30.71.109"
TEST_PATH = "./vanilla_test_images"

vanilla_detector = VanillaDetector(
    model=vanilla_model,
    path_to_images=PATH_TO_IMAGES,
    config_path=CONFIG_PATH,
    detection_controller=detection_controller
)

images = glob.glob(os.path.join(PATH_TO_IMAGES, "*"))
images = images[:1000]

for image in images:
    vanilla_detector.handle_image(image_path=image)

In [13]:
image_drawer._detection_controller._detections_of_image

{'/tf/mmk_smoke_detection/flow_data/172.30.71.109/frame_08-25-2021_07:54:22.jpg': [],
 '/tf/mmk_smoke_detection/flow_data/172.30.71.109/frame_08-25-2021_10:35:59.jpg': [{'place': '4',
   'detection': [190, 1131, 230, 1194],
   'score': 0.9999985694885254,
   'label': 'emission',
   'label_idx': 1,
   'line_thickness': 4},
  {'place': '22',
   'detection': [216, 30, 252, 81],
   'score': 0.9809531569480896,
   'label': 'emission',
   'label_idx': 1,
   'line_thickness': 4},
  {'place': '32',
   'detection': [656, 37, 689, 90],
   'score': 0.7960963249206543,
   'label': 'emission',
   'label_idx': 1,
   'line_thickness': 4}],
 '/tf/mmk_smoke_detection/flow_data/172.30.71.109/frame_08-24-2021_06:42:26.jpg': [{'place': '0',
   'detection': [0, 1131, 39, 1199],
   'score': 0.5979540944099426,
   'label': 'fire',
   'label_idx': 2,
   'line_thickness': 4},
  {'place': '1',
   'detection': [42, 1127, 83, 1199],
   'score': 0.5058128237724304,
   'label': 'fire',
   'label_idx': 2,
   'line_t

In [14]:
image_drawer.draw_random_images(number=100, dir_to_save=TEST_PATH, use_normalized_coordinates=False)

In [15]:
PATH_TO_SAVED_VANILLA_DETECTIONS_JSON = "vanilla_detections.json"
detection_controller.save(os.path.join(TEST_PATH, PATH_TO_SAVED_VANILLA_DETECTIONS_JSON))